# Problem Specification

In [1]:
inputRDD=sc.textFile("/data/students/bigdata_internet/lab2/word_frequency.tsv")
print("To look at some lines of the file we can use 'take' action: ")
print(inputRDD.take(3))
print("The number of words containing in the file is: ",inputRDD.count())

To look at some lines of the file we can use 'take' action: 
['have\t338996', 'bought\t46988', 'several\t19688']
The number of words containing in the file is:  339819


## Filter words starting with a specified prefix

In [3]:
PREFIX = "bo"
def checkPrefix(str):
    temp = str[0:len(PREFIX)]
    if temp == PREFIX:
        return True
    return False
filteredRDD = inputRDD.filter(checkPrefix)
print("Number of selected lines="+str(filteredRDD.count()))

Number of selected lines=1607


In [4]:
wordFreqMapRDD=filteredRDD.map(lambda line:line.split('\t'))
sortedFreqRDD = wordFreqMapRDD.sortBy(lambda e:int(e[1]),ascending=False)
print("Maxfreq="+str(sortedFreqRDD.take(1)[0][1]))
maxfreq =int(sortedFreqRDD.take(1)[0][1])

Maxfreq=47753


## Filter most frequent words

In [5]:
def checkFreq(elem):
    if int(elem[1])>0.8*maxfreq:
        return True
    return False
frequentWordRDD = wordFreqMapRDD.filter(checkFreq)
print(frequentWordRDD.collect())

[['bought', '46988'], ['box', '47753']]


## Count the remaining words and save the output

In [7]:
print("num of selected lines = "+str(frequentWordRDD.count()))

num of selected lines = 2


In [8]:
selectedWordRDD = frequentWordRDD.map(lambda elem:elem[0])
selectedWordRDD.saveAsTextFile("./lab2/ex1.3")

# Run the application in different ways

In [9]:
PREFIX = "ho"
filteredRDD = inputRDD.filter(checkPrefix)
print(filteredRDD.take(3))
print("number of selected lines="+str(filteredRDD.count()))

['hot\t32944', 'home\t17995', 'however\t12492']
number of selected lines=1519


**Python script:**

In [ ]:
from pyspark import SparkConf, SparkContext
import sys
conf = SparkConf().setAppName("My app")
sc = SparkContext(conf = conf)

inputRDD=sc.textFile("/data/students/bigdata_internet/lab2/word_frequency.tsv")
PREFIX =sys.argv[1]
def checkPrefix(str):
    temp = str[0:len(PREFIX)]
    if temp == PREFIX:
        return True
    return False
filteredRDD = inputRDD.filter(checkPrefix)
filteredRDD.saveAsTextFile(sys.argv[2])

**Command to run the script:**

s287513@jupyter-s287513:~/lab2$ spark-submit --master local --deploy-mode client problem2-2.py ho ./lab2/2-2

s287513@jupyter-s287513:~/lab2$ spark-submit --master yarn --deploy-mode client problem2-2.py ho ./lab2/2-2-re         

A1: When using "--master local", the execution of the script is faster than using "--master yarn". 
    The possible explanation is that the time used to transmit the script to yarn cluster is much more then the execution time of the script.

A2:Yes, I would cache filteredRDD.

# Bonus Task

In [15]:
inputRDD = sc.textFile("/data/students/bigdata_internet/lab2/finefoods_text.txt")
def toWord(line):
    words=[]
    for word in line.split(' '):
        if word != '':
            words.append(word)
    return words
wordRDD = inputRDD.flatMap(toWord)
print("Total word number="+str(wordRDD.count()))

Total word number=45444841


In [16]:
wordOnePairRDD = wordRDD.map(lambda word: (word,1))
wordFreqRDD = wordOnePairRDD.reduceByKey(lambda v1,v2:v1+v2)
print(wordFreqRDD.take(5))

[('have', 338996), ('bought', 46988), ('several', 19688), ('of', 792000), ('Vitality', 252)]
